Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

Описание данных 

- date – дата совершения события
- event - событие
- app_install – установка приложения
- app_start – открыть приложения
- register – зарегистрироваться 
- search – перейти на страницу поиска товаров (каталог)
- open_item – открыть товар
- choose_item – отправить товар в корзину
- tap_basket – перейти в корзину
- purchase – подтверждение покупки
- gender – пол пользователя
- os_name – платформа пользователя
- city – город пользователя
- device_id – идентификатор устройства пользователя
- urm_source – канал, с которого пришел пользователь

yandex-direct – Яндекс директ  
google_ads – реклама в Google  
vk_ads – реклама в ВК  
instagram_ads – реклама в instagram  
facebook_ads – реклама в facebook  
referral – акция «приведи друга»  
Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы  
purchase_sum – стоимость покупки (при совершении события ‘purchase’)

Обратите внимание на следующее:
- В выгрузке только уникальные действия пользователей за каждый день 
- Можно миновать стадию установки приложения, если оно было установлено ранее
- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

In [33]:
import pandas as pd
import numpy as np

In [34]:
df = pd.read_csv('/home/vitsli/DataAnalitycs/product_analitycs/KC_case_data.csv')

In [35]:
df.head(10)

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads
5,2020-01-01,app_start,NaN,android,5148476,male,Saint-Petersburg,-
6,2020-01-01,app_start,NaN,android,8383386,female,Saint-Petersburg,vk_ads
7,2020-01-01,app_start,NaN,android,8581267,female,Saint-Petersburg,google_ads
8,2020-01-01,app_start,NaN,ios,10144701,female,Moscow,-
9,2020-01-01,app_start,NaN,ios,10274354,female,Saint-Petersburg,instagram_ads


In [36]:
df.purchase_sum.isna().mean()

0.9485499831147961

In [37]:
df['date'] = pd.to_datetime(df['date'])

In [38]:
df.dtypes

date            datetime64[ns]
event                   object
purchase_sum           float64
os_name                 object
device_id                int64
gender                  object
city                    object
utm_source              object
dtype: object

In [39]:
df['month'] = df['date'].dt.month_name()

In [40]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,January
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,January
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,January


__Рассчитаем показатель MAU__

In [41]:
mau = df.groupby('month', as_index = False).agg({'device_id': 'nunique'}).rename(columns={'device_id':'mau'})
mau

,month,mau
0,February,75032
1,January,99161
2,March,74623


Наибольшее количество активных пользователей было в январе, посмотрим на этот месяц в разрезе активностей пользователей.

Посмотрим, какой из платных источников трафика привел нам наибольшее количество пользователей

In [45]:
(df.groupby('utm_source', as_index = False)
    .agg({'device_id': 'nunique'})
    .rename(columns={'device_id':'number_of_users'})
    .sort_values('number_of_users', ascending=False))

,utm_source,number_of_users
0,-,52273
6,yandex-direct,40712
2,google_ads,38096
5,vk_ads,34086
3,instagram_ads,31048
1,facebook_ads,25959
4,referal,15926


Из платных источников траффика наибольшее количество пользователей дал Яндекс.Директ

#### Рассчет CR из инсталла в покупку за первые семь дней (по когортам)


In [46]:
cohort = df[df.event == 'app_install'][['device_id', 'date']].rename(columns={'date': 'install_date'})

In [47]:
df = df.merge(cohort, how='left', on='device_id')

In [48]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,install_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January,2020-01-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,January,NaT
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,January,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,January,2020-01-01


In [49]:
cohort_size = (df.groupby('install_date', as_index=False)
                .agg({'device_id': 'nunique'})
                .rename(columns={'device_id': 'size'})
                .sort_values('size', ascending=False))
cohort_size.head()

,install_date,size
9,2020-01-10,13106
65,2020-03-06,6358
13,2020-01-14,5173
12,2020-01-13,4571
10,2020-01-11,4523


In [50]:
conversion = df[df.event == 'purchase']

In [51]:
conversion = conversion[(conversion.date - conversion.install_date <= pd.Timedelta('7 days'))]

In [52]:
conversion = (conversion.groupby('install_date', as_index=False)
                .agg({'device_id': 'nunique'})
                .rename(columns={'device_id': 'convert_size'}))

In [53]:
conversion = conversion.merge(cohort_size, how='inner', on='install_date')

In [54]:
conversion['cr'] = conversion['convert_size'] / conversion['size']

In [56]:
conversion.sort_values('cr', ascending=False).head()

,install_date,convert_size,size,cr
0,2020-01-01,1408,3579,0.393406
8,2020-01-09,558,1424,0.391854
14,2020-01-15,1650,4310,0.382831
13,2020-01-14,1973,5173,0.381403
1,2020-01-02,1186,3144,0.377226


Самый высокий CR показала когорта от 1 января 2020 года

#### Проанализируем на каком этапе воронки отваливается бОльшая часть зарегистрированнных клиентов

In [65]:
regdate = df[df.event == 'register'][['device_id', 'date']]\
            .sort_values('date')\
            .drop_duplicates('device_id')\
            .rename(columns={'date': 'regdate'})

In [67]:
regdate.head(10)

,device_id,regdate
2669658,294193,2020-01-01
2670356,21780,2020-01-01
2670357,2470180,2020-01-01
2670358,3735668,2020-01-01
2670359,11372938,2020-01-01
2670360,11296349,2020-01-01
2670361,7411279,2020-01-01
2670362,8778938,2020-01-01
2670363,31594,2020-01-01
2670364,314040,2020-01-01


In [68]:
df = df.merge(regdate, how='left', on='device_id')

In [70]:
df.head(10)

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,install_date,regstatus,regdate
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January,2020-01-01,False,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01,False,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,January,NaT,False,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,January,2020-01-01,False,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,January,2020-01-01,False,NaT
5,2020-01-01,app_start,NaN,android,5148476,male,Saint-Petersburg,-,January,2020-01-01,False,2020-01-24
6,2020-01-01,app_start,NaN,android,8383386,female,Saint-Petersburg,vk_ads,January,2020-01-01,False,2020-01-23
7,2020-01-01,app_start,NaN,android,8581267,female,Saint-Petersburg,google_ads,January,2020-01-01,False,2020-01-13
8,2020-01-01,app_start,NaN,ios,10144701,female,Moscow,-,January,2020-01-01,False,2020-01-01
9,2020-01-01,app_start,NaN,ios,10274354,female,Saint-Petersburg,instagram_ads,January,2020-01-01,False,NaT


In [71]:
reg_customers = df[(df.date > df.regdate)]

In [74]:
reg_customers.groupby('event', as_index=False).agg({'device_id':'nunique'})

,event,device_id
0,app_start,40991
1,choose_item,37926
2,purchase,24880
3,search,40482
4,tap_basket,34517


In [77]:
print('Конверсия в поиск из открытия приложения:', round(40482*100/40991, 2))

Конверсия в поиск из открытия приложения: 98.76


In [78]:
print('Конверсия из поиска в добавление товара в корзину:', round(37926*100/40482, 2))

Конверсия из поиска в добавление товара в корзину: 93.69


In [79]:
print('Конверсия из выбора товаров в переход в корзину', round(34517*100/37926, 2))

Конверсия из выбора товаров в переход в корзину 91.01


In [80]:
print('Конверсия из перехода в корзину в покупку', round(24880*100/34517,2))

Конверсия из перехода в корзину в покупку 72.08


In [41]:
df.groupby(['regstatus', 'event']).agg({'device_id': 'nunique'})

device_id
regstatus event                 
False     app_install     154597
          app_start       190884
          choose_item     155691
          purchase         70875
          search          184488
          tap_basket      125414
True      register         78310

#### Оценим, у какого из источников трафика наименьшая конверсия из первого открытия приложения в покупку

In [88]:
first_purchase = (
df[(df.event == 'purchase') & (df.utm_source != '-')]\
        .sort_values('date')\
        .drop_duplicates(subset='device_id', keep='first')\
        .groupby('utm_source')\
        .agg({'device_id': 'count'})
)    

first_run = (df[(df.event == 'app_start') & (df.utm_source != '-')]\
        .sort_values('date')\
        .drop_duplicates(subset='device_id', keep='first')\
        .groupby('utm_source')\
        .agg({'device_id': 'count'})
)

In [90]:
cr = first_purchase / first_run 
cr.rename(columns={'device_id': 'cr'})

,cr
utm_source,
facebook_ads,0.398453
google_ads,0.316611
instagram_ads,0.380584
referal,0.459947
vk_ads,0.397644
yandex-direct,0.311045


In [91]:
(df[df.event == 'purchase']
        .sort_values('date')
        .drop_duplicates(subset='device_id', keep='first')
        .groupby('utm_source')
        .agg({'purchase_sum': 'median'})
        .sort_values('purchase_sum'))

,purchase_sum
utm_source,
facebook_ads,389.0
google_ads,390.5
yandex-direct,392.5
vk_ads,393.0
instagram_ads,393.5
referal,395.5
-,398.5


#### Рассчитаем ROMI по платным каналам трафика.

Данные по затратам на рекламу:  
- Яндекс – 10 491 707 руб.
- Гугл – 10 534 878 руб.
- Фейсбук – 8 590 498 руб.
- Инстаграм – 8 561626 руб.
- ВК – 9 553 531руб.
- Расходы на реферальную программу: если пользователь приведет друга и последний совершит первую покупку, то оба получат по 100 рублей.

In [95]:
(romi_by_sources = df[df.utm_source != '-']
    .groupby('utm_source', as_index=False)
    .agg({'purchase_sum': 'sum'})
)
romi_by_sources

,utm_source,purchase_sum
0,facebook_ads,12249901.0
1,google_ads,12868276.0
2,instagram_ads,14546969.0
3,referal,8837044.5
4,vk_ads,16389652.5
5,yandex-direct,13915368.0


In [109]:
# рассчитаем количество реферральных клиентов, совершивших первую покупку
(df[df.utm_source == 'referal']
     .sort_values('date')
     .drop_duplicates(subset='device_id', keep='first')
     .device_id.nunique()
)

15926

In [111]:
print('Расходы на рефферальную программу составляют:', 200*15926.0)

Расходы на рефферальную программу составляют: 3185200.0


In [112]:
ls = [8590498.0, 10534878.0, 8561626.0, 3185200.0, 9553531.0, 10491707.0]
pd.Series(ls)

0     8590498.0
1    10534878.0
2     8561626.0
3     3185200.0
4     9553531.0
5    10491707.0
dtype: float64

In [114]:
romi_by_sources['advert_expenses'] = pd.Series(ls)
romi_by_sources['romi'] = ((romi_by_sources['purchase_sum'] - romi_by_sources['advert_expenses']) 
                           / romi_by_sources['advert_expenses'])

romi_by_sources.sort_values('romi', ascending=False)

,utm_source,purchase_sum,advert_expenses,romi
3,referal,8837044.5,3185200.0,1.774408
4,vk_ads,16389652.5,9553531.0,0.715560
2,instagram_ads,14546969.0,8561626.0,0.699090
0,facebook_ads,12249901.0,8590498.0,0.425983
5,yandex-direct,13915368.0,10491707.0,0.326321
1,google_ads,12868276.0,10534878.0,0.221493
